샘플용 파일 생성다항식

g1(x) = X^3 + X + 1

g2(x) = X^2 + 1

In [1]:
import numpy as np
from itertools import product

In [8]:
a = np.array([[1,2,3],[4,2,3]])

In [9]:
a[1]

array([4, 2, 3])

In [27]:
with open('', 'rb') as f:
    data = f.read()

In [29]:
pkt = data[0:8].hex()

In [30]:
pkt.encode()

b'784b924446506d46'

In [ ]:
class CCdecoder:
    '''
    뚝딱 짜본 CCDecode (rate = 1/2 전용)입니다
    '''
    def __init__(self, e_bit):
        self.e_bit = e_bit
        self.p1 = None
        self.p2 = None


In [14]:
import numpy as np
from itertools import product

def read_bits_from_file(filename, max_bits=None):
    bits = []
    with open(filename, "rb") as f:
        data = f.read()
        for byte in data:
            for i in range(8):
                bits.append((byte >> (7-i)) & 1)
    if max_bits is not None:
        bits = bits[:max_bits]
    return np.array(bits, dtype=np.uint8)

def conv_encode(u_bits, G, K):
    """rate-1/2 conv encoder → interleaved 출력 [y0,y1,y0,y1,...]"""
    state = 0
    y = []
    for b in u_bits:
        state = ((b << (K-1)) | state) & ((1<<K)-1)
        outs = []
        for g in G:
            acc = 0
            tmp = state
            for i in range(K):
                if (g >> i) & 1:
                    acc ^= (tmp >> (K-1-i)) & 1
            outs.append(acc)
        y.extend(outs)
        state >>= 1
    return np.array(y, dtype=np.uint8)

def bits_to_poly(mask_bits):
    val = 0
    for i,b in enumerate(mask_bits[::-1]):
        if b: val |= (1<<i)
    return val

def all_generators_K(K):
    """길이 K 생성다항식 후보 (MSB=1)"""
    cands = []
    for mask in product([0,1], repeat=K):
        if mask[0] == 1:
            cands.append(bits_to_poly(mask))
    return cands

def score_match(y_ref, y_obs):
    """interleaved 시퀀스 매치율"""
    L = min(len(y_ref), len(y_obs))
    return np.sum(y_ref[:L] == y_obs[:L]) / L

def blind_search_generators(y_obs, K=4, trial_len=200):
    """출력 비트열 하나만으로 rate-1/2 생성다항식 추정"""
    np.random.seed(0)
    u_trial = np.random.randint(0,2,trial_len)

    best = {"g1":None,"g2":None,"score":-1}
    gens = all_generators_K(K)

    for g1 in gens:
        for g2 in gens:
            if g1 == g2: continue
            y_ref = conv_encode(u_trial, [g1,g2], K)
            sc = score_match(y_ref, y_obs)
            if sc > best["score"]:
                best.update({"g1":g1,"g2":g2,"score":sc})
    return best

# -------------------
# 사용 예시
# -------------------

y_obs = read_bits_from_file("sample2_hello_world.bin")  # interleaved 출력 파일
K = 4   # constraint length

result = blind_search_generators(y_obs, K=K, trial_len=500)
print("추정된 생성다항식 g1,g2 (octal):", oct(result["g1"]), oct(result["g2"]))
print("매치율:", result["score"])


추정된 생성다항식 g1,g2 (octal): 0o14 0o12
매치율: 0.5572916666666666


In [16]:
bin(0o12)

'0b1010'